In [ ]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup

url_list = [
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
]

result = requests.get(url_list[0])

if result.status_code == 200:
    soup = BeautifulSoup(result.content, 'html.parser')

    products = soup.select('div.items__item.item-card')
    # pprint(products)

    for adv in products:
        name = adv.select('h2[class*=ItemTitle-module_item-title__]')
        price = adv.select('h2[class*=ItemTitle-module_item-title__]')
        print(name[0].get_text())
